In [90]:
!pip install squarify

In [0]:
import re
import string
from collections import Counter
import squarify
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

from bs4 import BeautifulSoup
import html as ihtml

import requests
import sqlite3

In [0]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [0]:
def extract_company(soup):
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-linl-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)

In [0]:
def extract_location(soup):
    locations = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            locations.append(div.find(name="span", attrs={"class":"location"}).text)
        except:
            locations.append("None")
    return(locations)

In [0]:
def extract_salary(soup):
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find(name="span", attrs={"class":"salaryText"}).text.replace("\n",""))
        except:
            salaries.append("None")
    return(salaries)

In [0]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [0]:
def extract_url(soup):
    urls = []
    for div1 in soup.find_all(name="div", attrs={"class":"row"}):
        for div2 in div1.find_all(name="div", attrs={"class":"title"}):
            for a in div2.find_all(name="a", href=True):
                urls.append(a['href'])
    return(urls)

def extract_desc(urls):
    descs = []
    for url in urls:
        full_url = "https://www.indeed.com" + url
        detail_page = requests.get(full_url)
        detail_soup = BeautifulSoup(detail_page.text, "html.parser")
        
        for div in detail_soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
            descs.append(div.text)
    return(descs)

In [0]:
def extract_count(soup):
    for div in soup.find_all(name="div", attrs={"id":"searchCountPages"}):
        temp_str = div.text.replace(" ", "")
        temp_count_str = re.search('of(.*)jobs', temp_str)
        count_str = re.sub('[^0-9]','', temp_count_str.group(1))
        count = int(count_str)
    return(count)

In [99]:
def extract_list(title_name, city_name, st_name):
    max_results = 10
    columns = ["city", "job_title", "company", "location", "salary", "description"]

    city_url = "https://www.indeed.com/jobs?q=" + title_name + \
               "&l=" + city_name + "%2C+" + st_name
    page = requests.get(city_url)
    soup = BeautifulSoup(page.text, "html.parser")
    max_counter = extract_count(soup)
    print(max_counter)

    job_title_list = []
    company_list = []
    location_list = []
    salary_list = []
    desc_list = []

    for start in range(0, max_results, 10):
        city_url = "https://www.indeed.com/jobs?q=" + title_name + \
                   "&l=" + city_name + "%2C+" + st_name + \
                   "&start=" + str(start)
        print(city_url)
        page = requests.get(city_url)
        soup = BeautifulSoup(page.text, "html.parser")

        job_title_list.extend(extract_job_title(soup))
        company_list.extend(extract_company(soup))
        location_list.extend(extract_location(soup))
        salary_list.extend(extract_salary(soup))
        add_urls = extract_url(soup)
        desc_list.extend(extract_desc(add_urls))

    return job_title_list, company_list, location_list, salary_list, desc_list

def job_db(title, city, st):   
    j_list, c_list, l_list, s_list, d_list = extract_list(title, city, st)     
    temp_df = pd.DataFrame(list(zip(j_list, c_list, l_list, s_list, d_list)), 
                          columns = ['job_title', 'company', 'location', 'salary', 'description'])
    return temp_df

temp1_df = job_db('data+scientist', 'San+Jose', 'CA')
temp2_df = job_db('data+scientist', 'San+Francisco', 'CA')
temp3_df = job_db('data+scientist', 'Seattle', 'WA')
temp4_df = job_db('data+scientist', 'Washington', 'DC')
temp5_df = job_db('data+scientist', 'New+York', 'NY')
temp6_df = job_db('data+scientist', 'Baltimore', 'MD')
temp7_df = job_db('data+scientist', 'Boulder', 'CO')
temp8_df = job_db('data+scientist', 'San+Diego', 'CA')
temp9_df = job_db('data+scientist', 'Denver', 'CO')
temp10_df = job_db('data+scientist', 'Huntsville', 'AL')
temp11_df = job_db('data+scientist', 'Colorado+Springs', 'CO')
temp12_df = job_db('data+scientist', 'Houston', 'TX')
temp13_df = job_db('data+scientist', 'Trenton', 'NJ')
temp14_df = job_db('data+scientist', 'Dallas', 'TX')
temp15_df = job_db('data+scientist', 'Columbus', 'OH')
temp16_df = job_db('data+scientist', 'Austin', 'TX')
temp17_df = job_db('data+scientist', 'Philadelphia', 'PA')
temp18_df = job_db('data+scientist', 'Durham', 'NC')
temp19_df = job_db('data+scientist', 'Raleigh', 'NC')
temp20_df = job_db('data+scientist', 'Atlanta', 'GA')


1314
https://www.indeed.com/jobs?q=data+scientist&l=San+Jose%2C+CA&start=0
939
https://www.indeed.com/jobs?q=data+scientist&l=San+Francisco%2C+CA&start=0
1107
https://www.indeed.com/jobs?q=data+scientist&l=Seattle%2C+WA&start=0
1173
https://www.indeed.com/jobs?q=data+scientist&l=Washington%2C+DC&start=0
824
https://www.indeed.com/jobs?q=data+scientist&l=New+York%2C+NY&start=0
163
https://www.indeed.com/jobs?q=data+scientist&l=Baltimore%2C+MD&start=0
161
https://www.indeed.com/jobs?q=data+scientist&l=Boulder%2C+CO&start=0
160
https://www.indeed.com/jobs?q=data+scientist&l=San+Diego%2C+CA&start=0
181
https://www.indeed.com/jobs?q=data+scientist&l=Denver%2C+CO&start=0
23
https://www.indeed.com/jobs?q=data+scientist&l=Huntsville%2C+AL&start=0
10
https://www.indeed.com/jobs?q=data+scientist&l=Colorado+Springs%2C+CO&start=0
98
https://www.indeed.com/jobs?q=data+scientist&l=Houston%2C+TX&start=0
92
https://www.indeed.com/jobs?q=data+scientist&l=Trenton%2C+NJ&start=0
252
https://www.indeed.com

In [100]:
indeed_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                       temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                       temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                       temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(indeed_df.shape)

(197, 5)


In [0]:
indeed_df = indeed_df.drop_duplicates(keep=False) 
indeed_df = indeed_df.reset_index(drop=True)

In [102]:
def convert_salary(sal_str, s_flag):
    
    sal_split = re.findall(r'\d+', sal_str.replace(",", ""))
    
    if len(sal_split) == 2:
        low_salary = int(sal_split[0])
        high_salary = int(sal_split[1])
    else:
        low_salary = None
        high_salary = None
    
    if s_flag == 'l':
        salary = low_salary
    else:
        salary = high_salary
    
    return salary

indeed_df['low_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'l'), axis=1)
indeed_df['high_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'h'), axis=1)
indeed_df = indeed_df.drop(columns=['salary'])

indeed_df.head()                                      

,job_title,company,location,description,low_salary,high_salary
0,"Siri - Data Scientist, Data Organization",Apple,"Santa Clara Valley, CA 95014","Summary\nPosted: Dec 6, 2019\nWeekly Hours: 40...",NaN,NaN
1,Data Scientist,SLEEP NUMBER,"San Jose, CA 95123 (Blossom Valley area)","Position Purpose:\nAs Data Scientist, you will...",NaN,NaN
2,Data Scientist,Spry Health,"Palo Alto, CA",Spry Health’s mission is to build the world’s ...,100000.0,135000.0
3,Data Scientist,PayPal,"San Jose, CA 95131 (North San Jose area)",This role is in the Global Resolutions and Pro...,NaN,NaN
4,Data Scientist,Ford Motor Company,"Palo Alto, CA",Data Scientist\n\n\nJob description:\n\n\nData...,NaN,NaN


In [103]:
def clean_text(text):
    text = text.replace('\\n', ' ')               # remove newline
    text = BeautifulSoup(text, "lxml").get_text() # remove html
    text = text.replace('/', ' ')                 # remove forward slashes
    text = re.sub(r'[^a-zA-Z ^0-9]', '', text)    # letters and numbers only
    text = text.lower()                           # lower case
    text = re.sub(r'(x.[0-9])', '', text)         # remove special characters
    return text

indeed_df['description'] = indeed_df.apply(lambda x: clean_text(x['description']), axis=1)

indeed_df.head()

,job_title,company,location,description,low_salary,high_salary
0,"Siri - Data Scientist, Data Organization",Apple,"Santa Clara Valley, CA 95014",summaryposted dec 6 2019weekly hours 40role nu...,NaN,NaN
1,Data Scientist,SLEEP NUMBER,"San Jose, CA 95123 (Blossom Valley area)",position purposeas data scientist you will pla...,NaN,NaN
2,Data Scientist,Spry Health,"Palo Alto, CA",spry healths mission is to build the worlds la...,100000.0,135000.0
3,Data Scientist,PayPal,"San Jose, CA 95131 (North San Jose area)",this role is in the global resolutions and pro...,NaN,NaN
4,Data Scientist,Ford Motor Company,"Palo Alto, CA",data scientistjob descriptiondata scientist po...,NaN,NaN


In [104]:
indeed_df['description'][0]

'summaryposted dec 6 2019weekly hours 40role number200125352would you like to play a part in the next revolution in humancomputer interaction contribute to a product that is redefining mobile and desktop computing and work with the people who built the intelligent assistant that helps millions of people get things done  just by askingthe vision for the siri data organization is to improve siri by using data as the voice of our customers within this organization the mission of the analytics team is to inform the evolution of siri through measurement and analysis of the user experience as a data scientist on the team you will be an ambassador of analytics to language understanding product and engineering teams with the ultimate purpose of improving the siri experience for apple customerskey qualificationsyou think about language and complex systems in terms of statistical distributions and have a big enough toolbox to know how to find patterns in data identify targets for performance and

In [105]:
indeed_df['description'][4]

'data scientistjob descriptiondata scientist positions offered by ford motor company palo alto ca build machine learning models to solve business problems perform data acquisition and cleaning data exploration hypothesis formation and testing experimental design and custom software developmentminimum requirementsmasters or foreign equivalent degree in industrial engineering computer science or a related field3 years of experience in the position offered or 3 years of experience in automotive industry supporting business units with machine learning and deep learning techniques3 years of experience developing machine learning models with a focus on text analytics applications with natural language processing techniques including parsing word embeddings clustering cosine distance similarity metrics language modeling and ontologies3 years of experience performing hypothesis testing and inference probability distributions confidence intervals and pvalues3 years of experience working in hado

In [0]:
# convert to sqlite3
conn = sqlite3.connect('techsearch.sqlite3')
curs = conn.cursor()
curs.execute('drop table if exists listings')
indeed_df.to_sql('listings', con=conn)